## 중기 기상 데이터 크롤링 및 좌표 추가 분석

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime, timedelta
from geopy.geocoders import Nominatim

n_values = [1, 7, 9, 10, 13, 16, 22, 28, 34, 40]  # 지정된 특정 행 번호

def get_weather_info_mid_term():
    # Selenium WebDriver 설정
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # URL 접속
    url = "https://www.weather.go.kr/w/weather/forecast/mid-term.do"
    driver.get(url)
    time.sleep(5)  # 페이지 로드 대기

    weather_data = []  # 데이터 저장 리스트

    # 오늘 날짜 계산
    today = datetime.today()
    day_index = [(today + timedelta(days=i)).strftime("%d일") for i in range(4, 8)]

    # 모든 행 처리 (n_values 포함)
    rows = driver.find_elements(By.CSS_SELECTOR, "div.tab-menu-cont > div:nth-child(9) > table > tbody > tr")
    row_count = len(rows)

    for n in range(1, row_count + 1):
        try:
            # 지역명 추출
            region_selector = f"div.tab-menu-cont > div:nth-child(9) > table > tbody > tr:nth-child({n}) > td.midterm-city > span"
            region = driver.find_element(By.CSS_SELECTOR, region_selector).text.strip()

            # day_column 동적 설정
            if n in n_values:
                day_columns = [3, 4, 5, 6]  # n_values에 포함된 경우
            else:
                day_columns = [2, 3, 4, 5]  # 그 외의 경우

            # 날짜별 데이터 추출
            temp_data = {"지역": region}
            for day, day_column in zip(day_index, day_columns):
                # 최고 기온 추출
                high_temp_selector = f"div.tab-menu-cont > div:nth-child(9) > table > tbody > tr:nth-child({n}) > td:nth-child({day_column}) > span.tmx"
                high_temp = driver.find_element(By.CSS_SELECTOR, high_temp_selector).text.strip()

                # 최저 기온 추출
                low_temp_selector = f"div.tab-menu-cont > div:nth-child(9) > table > tbody > tr:nth-child({n}) > td:nth-child({day_column}) > span.tmn"
                low_temp = driver.find_element(By.CSS_SELECTOR, low_temp_selector).text.strip()

                # 최고/최저 기온 결합하여 저장
                temp_data[day] = f"{low_temp}/{high_temp}"

            weather_data.append(temp_data)
        except Exception as e:
            print(f"Error processing row {n}: {e}")
            continue  # 에러 발생 시 건너뜀

    driver.quit()  # 브라우저 종료
    return weather_data

def add_coordinates(data):
    # Geolocator 설정
    geolocator = Nominatim(user_agent="weather_locator")
    for entry in data:
        try:
            if entry["지역"] == "고창":
                entry["위도"] = 35.4358216
                entry["경도"] = 126.7020806
            elif entry["지역"] == "통영":
                entry["위도"] = 34.8544227
                entry["경도"] = 128.433182
            else:
                location = geolocator.geocode(entry["지역"])
                if location:
                    entry["위도"] = location.latitude
                    entry["경도"] = location.longitude
                else:
                    entry["위도"] = None
                    entry["경도"] = None
        except Exception as e:
            print(f"Error fetching coordinates for {entry['지역']}: {e}")
            entry["위도"] = None
            entry["경도"] = None
    return data

def save_to_excel(data, file_name="mid_term_weather_data_with_coordinates.xlsx"):
    # 데이터프레임 생성
    df = pd.DataFrame(data)
    # 엑셀 파일로 저장
    df.to_excel(file_name, index=False)
    print(f"데이터가 {file_name}에 저장되었습니다.")

# 실행 및 출력
weather_data = get_weather_info_mid_term()
weather_data_with_coordinates = add_coordinates(weather_data)
print(weather_data_with_coordinates)
save_to_excel(weather_data_with_coordinates)

[{'지역': '서울', '01일': '-4/6', '02일': '-3/3', '03일': '-5/2', '04일': '-5/2', '위도': 37.5666791, '경도': 126.9782914}, {'지역': '인천', '01일': '-3/5', '02일': '-2/3', '03일': '-4/1', '04일': '-4/2', '위도': 37.456, '경도': 126.7052}, {'지역': '수원', '01일': '-5/6', '02일': '-4/3', '03일': '-5/2', '04일': '-6/2', '위도': 37.2662775, '경도': 127.0004098}, {'지역': '파주', '01일': '-9/5', '02일': '-8/3', '03일': '-10/1', '04일': '-9/2', '위도': 37.8150577, '경도': 126.7924283}, {'지역': '이천', '01일': '-7/5', '02일': '-5/4', '03일': '-7/2', '04일': '-8/2', '위도': 37.2642411, '경도': 127.4422242}, {'지역': '평택', '01일': '-5/6', '02일': '-3/4', '03일': '-5/1', '04일': '-6/1', '위도': 36.9908233, '경도': 127.0853183}, {'지역': '춘천', '01일': '-8/4', '02일': '-7/4', '03일': '-8/2', '04일': '-9/1', '위도': 37.8845604, '경도': 127.716664}, {'지역': '원주', '01일': '-6/4', '02일': '-4/4', '03일': '-6/2', '04일': '-7/2', '위도': 37.3164083, '경도': 127.9228206}, {'지역': '강릉', '01일': '-1/7', '02일': '0/6', '03일': '-2/4', '04일': '-3/5', '위도': 37.76452, '경도': 128.8993979}, {'지역': '대전

## 4일 뒤 전국 빙결 예상 분포도

In [5]:
import pandas as pd
import folium
import webbrowser
import os
from datetime import datetime, timedelta
import pickle

# pickle 파일 로드
with open("temperature_data.pkl", "rb") as f:
    data = pickle.load(f)

temp_00 = data["temp_00"] #저위험
temp_01 = data["temp_01"] #위험

m = folium.Map(location=[36.5, 127.5], zoom_start=7)  # 대한민국 중심 좌표
def create_map(data):
    # Folium 지도 초기화   

    # 오늘 날짜 기준으로 +3일 날짜 계산
    target_date = (datetime.today() + timedelta(days=4)).strftime("%d일")

    for _, row in data.iterrows():
        try:
            region = row["지역"]
            lat = row["위도"]
            lon = row["경도"]
            
            # 동적으로 결정된 날짜에 해당하는 데이터 추출
            temp = row[target_date].split('/')[0]  # 최저 온도만 추출
            
            # 색상 결정
            temp = int(temp)
            if temp < temp_01:
                color = "red"
            elif temp_01 <= temp < temp_00:
                color = "green"
            elif temp >= temp_00:
                color = "blue"

            # 마커 추가
            folium.CircleMarker(
                location=[lat, lon],
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{region}: {row[target_date]}",
            ).add_to(m)
            
        except Exception as e:
            print(f"Error adding marker for {row['지역']}: {e}")
            continue
 
# 실행
file_name = "mid_term_weather_data_with_coordinates.xlsx"
data = pd.read_excel(file_name)
create_map(data)
m


## 5일 뒤 전국 빙결 예상 분포도

In [4]:
import pandas as pd
import folium
import webbrowser
import os
from datetime import datetime, timedelta
import pickle

# pickle 파일 로드
with open("temperature_data.pkl", "rb") as f:
    data = pickle.load(f)

temp_00 = data["temp_00"] #저위험
temp_01 = data["temp_01"] #위험

m = folium.Map(location=[36.5, 127.5], zoom_start=7)  # 대한민국 중심 좌표
def create_map(data):
    # Folium 지도 초기화   

    # 오늘 날짜 기준으로 +3일 날짜 계산
    target_date = (datetime.today() + timedelta(days=5)).strftime("%d일")

    for _, row in data.iterrows():
        try:
            region = row["지역"]
            lat = row["위도"]
            lon = row["경도"]
            
            # 동적으로 결정된 날짜에 해당하는 데이터 추출
            temp = row[target_date].split('/')[0]  # 최저 온도만 추출
            
            # 색상 결정
            temp = int(temp)
            if temp < temp_01:
                color = "red"
            elif temp_01 <= temp < temp_00:
                color = "green"
            elif temp >= temp_00:
                color = "blue"

            # 마커 추가
            folium.CircleMarker(
                location=[lat, lon],
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{region}: {row[target_date]}",
            ).add_to(m)
            
        except Exception as e:
            print(f"Error adding marker for {row['지역']}: {e}")
            continue
 
# 실행
file_name = "mid_term_weather_data_with_coordinates.xlsx"
data = pd.read_excel(file_name)
create_map(data)
m

## 6일 뒤 전국 빙결 예상 분포도

In [7]:
import pandas as pd
import folium
import webbrowser
import os
from datetime import datetime, timedelta
import pickle

# pickle 파일 로드
with open("temperature_data.pkl", "rb") as f:
    data = pickle.load(f)

temp_00 = data["temp_00"] #저위험
temp_01 = data["temp_01"] #위험

m = folium.Map(location=[36.5, 127.5], zoom_start=7)  # 대한민국 중심 좌표
def create_map(data):
    # Folium 지도 초기화   

    # 오늘 날짜 기준으로 +3일 날짜 계산
    target_date = (datetime.today() + timedelta(days=6)).strftime("%d일")

    for _, row in data.iterrows():
        try:
            region = row["지역"]
            lat = row["위도"]
            lon = row["경도"]
            
            # 동적으로 결정된 날짜에 해당하는 데이터 추출
            temp = row[target_date].split('/')[0]  # 최저 온도만 추출
            
            # 색상 결정
            temp = int(temp)
            if temp < temp_01:
                color = "red"
            elif temp_01 <= temp < temp_00:
                color = "green"
            elif temp >= temp_00:
                color = "blue"

            # 마커 추가
            folium.CircleMarker(
                location=[lat, lon],
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{region}: {row[target_date]}",
            ).add_to(m)
            
        except Exception as e:
            print(f"Error adding marker for {row['지역']}: {e}")
            continue
 
# 실행
file_name = "mid_term_weather_data_with_coordinates.xlsx"
data = pd.read_excel(file_name)
create_map(data)
m

## 7일 뒤 전국 빙결 예상 분포도

In [2]:
import pandas as pd
import folium
import webbrowser
import os
from datetime import datetime, timedelta
import pickle

# pickle 파일 로드
with open("temperature_data.pkl", "rb") as f:
    data = pickle.load(f)

temp_00 = data["temp_00"] #저위험
temp_01 = data["temp_01"] #위험

m = folium.Map(location=[36.5, 127.5], zoom_start=7)  # 대한민국 중심 좌표
def create_map(data):
    # Folium 지도 초기화   

    # 오늘 날짜 기준으로 +3일 날짜 계산
    target_date = (datetime.today() + timedelta(days=7)).strftime("%d일")

    for _, row in data.iterrows():
        try:
            region = row["지역"]
            lat = row["위도"]
            lon = row["경도"]
            
            # 동적으로 결정된 날짜에 해당하는 데이터 추출
            temp = row[target_date].split('/')[0]  # 최저 온도만 추출
            
            # 색상 결정
            temp = int(temp)
            if temp < temp_01:
                color = "red"
            elif temp_01 <= temp < temp_00:
                color = "green"
            elif temp >= temp_00:
                color = "blue"

            # 마커 추가
            folium.CircleMarker(
                location=[lat, lon],
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{region}: {row[target_date]}",
            ).add_to(m)
            
        except Exception as e:
            print(f"Error adding marker for {row['지역']}: {e}")
            continue
 
# 실행
file_name = "mid_term_weather_data_with_coordinates.xlsx"
data = pd.read_excel(file_name)
create_map(data)
m